In [13]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


ImportError: dlopen(/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tfe.so, 0x0002): Library not loaded: @rpath/_pywrap_tensorflow_internal.so
  Referenced from: <8D36263D-A690-33C2-92FC-5C2A03029529> /Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tfe.so
  Reason: tried: '/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utfe.so___Utensorflow/_pywrap_tensorflow_internal.so' (no such file), '/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal_Umacos___Utensorflow_Spython/_pywrap_tensorflow_internal.so' (no such file), '/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so' (no such file), '/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../_py

In [14]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/Mehrdadi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Mehrdadi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
data_file = open('intents.json')
data = json.load(data_file)

Identifying Feature and Target for the NLP Model

In [16]:
words = []
classes = []
data_X = []
data_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(pattern)
        data_y.append(intent["tag"])
        
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes =sorted(set(classes))

In [17]:
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate (data_X):
    bow = []
    text = lemmatizer. lemmatize(doc. lower ())
    for word in words:
        bow. append(1) if word in text else bow. append (0)

    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded Boll and associated classes to training
    training. append ([bow, output_row])


random.shuffle(training)
training = np.array (training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np. array(list(training[:, 1]))

In [18]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len (train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])

print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

NameError: name 'Sequential' is not defined

In [ ]:
def clean_text (text):
    tokens = nltk. word_tokenize (text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words (text, vocab):
    tokens = clean_text (text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate (vocab):
            if word == w:
                bow[idx] = 1
    return np.array (bow)


def pred_class (text, vocab, labels):
    bow = bag_of_words (text, vocab)
    result = model. predict (np.array ([bow]))[0] #extracting probabilities
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate (result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in dec
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]]) #Contains labels (tags) for highest probability 
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random. choice(i["responses"])
                break
    return result